In [14]:
import pandas as pd
import sqlite3
import requests
import re

In [15]:
def clean_data(df):
    """Clean the orders data"""
    # Clean dollar amounts
    df['$ sale'] = df['$ sale'].str.replace('$', '').str.replace(',', '').str.strip().astype(float)
    
    # Clean state abbreviations (convert to uppercase and fix inconsistencies)
    df['state'] = df['state'].str.upper().str.strip()
    
    # Clean zip codes
    df['Zip'] = df['Zip'].astype(str).str.strip()

    # dropping the duplicates from order data assuming that order numbers can not be duplicated
    df.drop_duplicates(subset=['order_number'], inplace=True)
    
    # Rename columns to match our database schema
    df = df.rename(columns={
        'date': 'order_date',
        '$ sale': 'order_amount',
        'Zip': 'zip_code'
    })
    
    return df

In [16]:
def setup_database():
    """Initialize the database"""
    conn = sqlite3.connect('sales_orders.db')
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS orders (
            order_number TEXT PRIMARY KEY,
            order_date TEXT,
            city TEXT,
            state TEXT,
            zip_code TEXT,
            order_amount REAL,
            ip_address TEXT,
            processed INTEGER DEFAULT 0
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS ip_cache (
            ip_address TEXT PRIMARY KEY,
            city TEXT,
            state TEXT,
            zip_code TEXT
        )
    ''')
    
    conn.commit()
    return conn

In [17]:
def load_orders(conn, orders_file):
    """Load and clean orders data"""
    df = pd.read_csv(orders_file)
    df = clean_data(df)
    
    # Get existing order numbers
    existing_orders = pd.read_sql('SELECT DISTINCT order_number FROM orders', conn)
    existing_numbers = existing_orders['order_number'].tolist()
    
    # Filter out duplicates
    new_orders = df[~df['order_number'].isin(existing_numbers)]
    
    if not new_orders.empty:
        # Insert in chunks to handle potential errors
        chunk_size = 100
        for i in range(0, len(new_orders), chunk_size):
            chunk = new_orders.iloc[i:i + chunk_size]
            try:
                chunk.to_sql('orders', conn, if_exists='append', index=False)
                print(f"Added {len(chunk)} new orders (chunk {i//chunk_size + 1})")
            except sqlite3.IntegrityError as e:
                print(f"Error inserting chunk {i//chunk_size + 1}: {str(e)}")
    else:
        print("No new orders to add")

In [18]:
def get_location_from_ip(ip):
    """Get location from IP using free API"""
    if not ip or pd.isna(ip) or ip.strip() == '':
        return None
        
    try:
        response = requests.get(f'http://ip-api.com/json/{ip}?fields=status,message,city,region,zip')
        data = response.json()
        
        if data.get('status') == 'success':
            return {
                'city': data.get('city'),
                'state': data.get('region'),
                'zip_code': data.get('zip')
            }
    except Exception as e:
        print(f"Error processing IP {ip}: {str(e)}")
    
    return None

In [19]:
def process_ip_addresses(conn, ip_file):
    """Ultra-fast IP processing with bulk operations"""
    # 1. Load data with optimized parameters
    ip_df = pd.read_csv(ip_file, usecols=['order_number', 'ip_address'])
    
    # 2. Fast filtering of invalid IPs
    mask = ip_df['ip_address'].notna() & (ip_df['ip_address'].str.strip() != '')
    ip_df = ip_df[mask].copy()
    ip_df['ip_address'] = ip_df['ip_address'].str.strip()

    ip_df['ip_address'] = ip_df['ip_address'].str.rstrip(':')
    
    if ip_df.empty:
        print("No valid IP addresses to process")
        return

    # 3. Get cached IPs in one query (faster than DISTINCT)
    cached_ips = pd.read_sql(
        'SELECT ip_address FROM ip_cache GROUP BY ip_address', 
        conn
    )['ip_address'].values
    
    # 4. Filter out cached IPs using numpy (much faster than isin)
    import numpy as np
    cached_set = set(cached_ips)
    uncached_mask = ~np.isin(ip_df['ip_address'].values, list(cached_set))
    uncached_df = ip_df[uncached_mask]
    
    if uncached_df.empty:
        print("All IPs already cached")
        return

    # 5. Process unique IPs in bulk
    unique_ips = uncached_df['ip_address'].unique()
    ip_to_orders = uncached_df.groupby('ip_address')['order_number'].apply(list).to_dict()
    
    # 6. Prepare bulk inserts and updates
    cache_inserts = []
    order_updates = []
    
    for ip in unique_ips:
        location = get_location_from_ip(ip)
        if location and location.get('zip_code'):
            # Prepare cache insert
            cache_inserts.append((
                ip,
                location['city'],
                location['state'],
                location['zip_code']
            ))
            
            # Prepare order updates for all orders with this IP
            for order_num in ip_to_orders[ip]:
                order_updates.append((
                    location['city'],
                    location['state'],
                    location['zip_code'],
                    ip,
                    order_num
                ))
    
    # 7. Execute bulk operations
    cursor = conn.cursor()
    
    print('Started Inserting')
    # Bulk insert to cache
    if cache_inserts:
        cursor.executemany('''
            INSERT INTO ip_cache (ip_address, city, state, zip_code)
            VALUES (?, ?, ?, ?)
        ''', cache_inserts)
    
    # Bulk update orders
    if order_updates:
        cursor.executemany('''
            UPDATE orders 
            SET city = ?, state = ?, zip_code = ?, ip_address = ?, processed = 1
            WHERE order_number = ? AND processed = 0
        ''', order_updates)
    
    conn.commit()
    
    print(f"Processed {len(unique_ips)} IPs, updated {len(order_updates)} orders")

In [20]:
def export_enriched_orders(conn, output_file):
    """Export processed orders to CSV"""
    df = pd.read_sql('''
        SELECT 
            order_number,
            order_date,
            city,
            state,
            zip_code,
            order_amount
        FROM orders 
        WHERE processed = 1
    ''', conn)
    
    df.to_csv(output_file, index=False)
    print(f"Exported {len(df)} orders to {output_file}")

In [21]:
conn = setup_database()

In [22]:
load_orders(conn, 'orders_file.csv')

No new orders to add


In [23]:
process_ip_addresses(conn, 'ip_addresses_test.csv')

Started Inserting
Processed 200 IPs, updated 194 orders


In [24]:
export_enriched_orders(conn, 'enriched_orders.csv')

Exported 194 orders to enriched_orders.csv


In [25]:
def generate_sales_report(conn, state, year, output_file):
    """Generate quarterly sales report"""
    # Get orders data
    query = '''
        SELECT 
            order_date,
            city,
            order_amount
        FROM orders
        WHERE state = ? AND substr(order_date, 1, 4) = ?
            AND city IS NOT NULL AND city != ''
    '''
    df = pd.read_sql(query, conn, params=(state, str(year)))
    
    if df.empty:
        print(f"No data found for {state} in {year}")
        return
    
    # Process dates and quarters
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['quarter'] = df['order_date'].dt.quarter
    
    # Group by city and quarter
    grouped = df.groupby(['city', 'quarter'])['order_amount'].sum().unstack()
    
    # Create report in the required format
    report_data = []
    
    # Add header
    report_data.append([f"{year} State: {state}", "", ""])
    
    for q in range(1, 5):
        report_data.append([f"Q{q}", "City", "Total Sales"])
        
        if q in grouped.columns:
            quarter_data = grouped[q].reset_index()
            quarter_data.columns = ['City', 'Total Sales']
            quarter_data = quarter_data.sort_values('City')
            
            for _, row in quarter_data.iterrows():
                report_data.append(["", row['City'], row['Total Sales']])
        else:
            print(f"No data for Q{q}")
        
        report_data.append(["", "", ""])  # Empty row between quarters
    
    # Create DataFrame and export to Excel
    report_df = pd.DataFrame(report_data, columns=["A", "B", "C"])
    
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        report_df.to_excel(
            writer, 
            sheet_name="Sales Report", 
            index=False, 
            header=False
        )
        
        # Adjust column widths
        worksheet = writer.sheets["Sales Report"]
        worksheet.column_dimensions['A'].width = 15
        worksheet.column_dimensions['B'].width = 15
        worksheet.column_dimensions['C'].width = 15
    
    print(f"Report generated: {output_file}")

In [29]:
generate_sales_report(conn, 'IL', 2024, 'IL_state_sales_report_2024.xlsx')

No data for Q4
Report generated: IL_state_sales_report_2024.xlsx


In [1]:
pip freeze

asttokens==3.0.0
certifi==2025.1.31
charset-normalizer==3.4.1
comm==0.2.2
debugpy==1.8.13
decorator==5.2.1
et_xmlfile==2.0.0
exceptiongroup==1.2.2
executing==2.2.0
idna==3.10
ipykernel==6.29.5
ipython==8.34.0
jedi==0.19.2
jupyter_client==8.6.3
jupyter_core==5.7.2
matplotlib-inline==0.1.7
nest-asyncio==1.6.0
numpy==2.2.4
openpyxl==3.1.5
packaging==24.2
pandas==2.2.3
parso==0.8.4
pexpect==4.9.0
platformdirs==4.3.7
prompt_toolkit==3.0.50
psutil==7.0.0
ptyprocess==0.7.0
pure_eval==0.2.3
Pygments==2.19.1
python-dateutil==2.9.0.post0
pytz==2025.2
pyzmq==26.3.0
requests==2.32.3
six==1.17.0
stack-data==0.6.3
tornado==6.4.2
traitlets==5.14.3
typing_extensions==4.13.0
tzdata==2025.2
urllib3==2.3.0
wcwidth==0.2.13
Note: you may need to restart the kernel to use updated packages.
